In [ ]:
import itertools
import logging
import networkx as nx
import pprint

logger = logging.getLogger(__name__)

# Categories
types = {'pieces' : [500, 750, 1000, 1250, 1500, 1750],
         'years' : [1981, 1982, 1992, 1995, 1998, 1999],
         'companies' : ["Buralde", "Chaurd", "Furoth", "Garroda", "Irycia", "Rynir"],
         'themes' : ["autumn leaves", "city skyline", "coral reef", "football", "outer space", "postage stamp"]}

def build_graph(categories):
    all_items = []
    for v in types.values():
        all_items += v
    
    G = nx.Graph()
    for k, v in types.items():
        G.add_nodes_from(v, label=k)
    
    # Add edges
    lists = [v for v in types.values()]
    
    # Add edges between nodes from different lists, but no edges within the same list
    for i, list1 in enumerate(lists):
        for j, list2 in enumerate(lists):
            if i < j:  # To avoid edges within the same list and duplicating edges
                for node1, node2 in itertools.product(list1, list2):
                    G.add_edge(node1, node2)

    return G, all_items

G, all_items = build_graph(types)
print(G.number_of_edges())

In [ ]:
def node_type(n):
    for t, vs in types.items():
        if n in vs:
            return t

def neighbors(n, ntype = None):
    if ntype:
        assert n not in types[ntype]
        return [n for n in G.adj[n] if n in types[ntype]]
    else:
        return [n for n in G.neighbors(n)]

def neighbors_by_type(n):
    ntype = node_type(n)
    adj = {}
    for t in types.keys():
        if t == ntype:
            continue
        adj[t] = neighbors(n, t)
    return adj

def has_one_edge(n, ntype):
    # print(f"Checking edge count on {n} of type '{ntype}'")
    count = count_edges_per_type(n)
    if count[ntype] == 0:
        print(n, neighbors_by_type(n))
    assert count[ntype] != 0
    return count[ntype] == 1

def count_edges_per_type(n):
    adj = neighbors_by_type(n)
    return { k : len(v) for k,v in adj.items()}

def remove_edge(a, b) -> bool:
    assert a in all_items
    assert b in all_items
    try:
        G.remove_edge(a, b)
        print(f"Removed {a}<->{b}")
        return True
    except nx.NetworkXError:
        return False

def piece_delta(lesser, greater, delta):
    remove_edge(lesser, greater)
    pieces = types['pieces']
    for p in pieces:
        p_plus_delta = p + delta
        p_minus_delta = p - delta
        # if p + delta isn't in greater, then p can't be in lesser
        if p_plus_delta in pieces and p_plus_delta not in neighbors(greater, "pieces"):
            remove_edge(lesser, p)
        if p < min(neighbors(lesser, "pieces")) + delta:
            remove_edge(greater, p)
        # if p - delta isn't in lesser, then p can't be in greater
        if p_minus_delta in pieces and p_minus_delta not in  neighbors(lesser, "pieces"):
            remove_edge(greater, p)
        if p > max(neighbors(greater, "pieces")) - delta:
            remove_edge(lesser, p)
        
def either_or(obj, pair):
    # Example:
    #   outerspace == (Irycia ^ 1250)
    # If outerspace only has the 1250 edge, then it can't have the Irycia edge
    # If outerspace only has the Irycia edge, then it can't have the 1250 edge
    # Also:
    # If Irycia has only one edge to type(outerspace) and not outerspace, then
    #   1250 belongs to outerspace. All other edges of type(1250) can be removed
    # If 1250 has only one edge to type(outerspace) and not outerspace, then
    #   Irycia belongs to outerspace. All other edges of type(Irycia) can be removed

    assert len(pair) == 2
    obj_adj = neighbors_by_type(obj)
    obj_type = node_type(obj)
    p1, p2 = pair
    p1_adj = neighbors_by_type(p1)
    p1_type = node_type(p1)
    p2_adj = neighbors_by_type(p2)
    p2_type = node_type(p2)
    
    # Check if p1 is a neighbor of obj. If not, then p2 belongs to obj.
    # Remove all edges that are not p2 of type(p2) from obj
    if p1 not in obj_adj[p1_type]:
        isolate_edge(obj, p2)
    # Check if p2 is a neighbor of obj. If not, then p1 belongs to obj.
    # Remove all edges that are not p1 of type(p1) from obj
    if p2 not in obj_adj[p2_type]:
        isolate_edge(obj, p1)

    # Check if obj has one edge of type(p1) and is p1
    obj_p1_type_neighbors = neighbors(obj, p1_type)
    if len(obj_p1_type_neighbors) == 1 and p1 in obj_p1_type_neighbors:
        remove_edge(obj, p2)
    # Check if obj has one edge of type(p2) and is p2
    obj_p2_type_neighbors = neighbors(obj, p2_type)
    if len(obj_p2_type_neighbors) == 1 and p1 in obj_p2_type_neighbors:
        remove_edge(obj, p1)

    # if p1 has one neighbor of type(obj) and not obj, then
    #   p2 belongs to obj
    p1_neighbors = neighbors(p1, obj_type)
    if len(p1_neighbors) == 1 and obj not in p1_neighbors:
        isolate_edge(obj, p2)
    # if p2 has one neighbor of type(obj) and not obj, then
    #   p1 belongs to obj
    p2_neighbors = neighbors(p2, obj_type)
    if len(p2_neighbors) == 1 and obj not in p2_neighbors:
        isolate_edge(obj, p1)

def share_info(node1, node2):
    print(f"Sharing info between {node1} & {node2}")
    node1_type = node_type(node1)
    node2_type = node_type(node2)
    # for the types that aren't this type, we want to get neighbors,
    # find the symmetric difference, and eliminate those edges from each other
    for t in types.keys():
        if t == node1_type:
            unique_v_of_t = set([node1]).symmetric_difference(neighbors(node2, t))
        elif t == node2_type:
            unique_v_of_t = set(neighbors(node1, t)).symmetric_difference([node2])
        else:
            unique_v_of_t = set(neighbors(node1, t)).symmetric_difference(neighbors(node2, t))
        # print(f"unique '{t}' values: ", unique_v_of_t)
        for v in unique_v_of_t:
            if t != node1_type:
                remove_edge(node1, v)
            if t != node2_type:
                remove_edge(node2, v)

def isolate_edge(node1, node2):
    # There's a guaranteed edge between node1 and node2
    # Eliminate all type(node1) edges that are not node1 from node2
    # Eliminate all type(node2) edges that are not node2 from node1
    type1 = node_type(node1)
    type2 = node_type(node2)
    assert node1 in types[type1]
    assert node2 in types[type2]
    not_node1 = set(types[type1]).difference([node1])
    # print(f"'{type1}' node: {node1}, not node: {not_node1}")
    for n in not_node1:
        remove_edge(n, node2)
    not_node2 = set(types[type2]).difference([node2])
    # print(f"'{type2}' node: {node2}, not node: {not_node2}")
    for n in not_node2:
        remove_edge(n, node1)

print("--- Begin Rules ---")
print(f"--- Edges: {G.number_of_edges()}")
print("+ Rule 1 +")
#   outerspace == (Irycia ^ 1250)
either_or("outer space", ("Irycia", 1250))

print("+ Rule 2 +")
# Clue states: The jigsaw puzzle made by Rynir has "somewhat more" than
# the puzzle with the postage stamp theme. "somewhat" is subjective
#   Rynir pieces > postage stamp pieces
# piece_delta("postage stamp", "Rynir", 10)
remove_edge("Rynir", "postage stamp")
remove_edge("Rynir", min(neighbors("postage stamp", "pieces")))
remove_edge("postage stamp", max(neighbors("Rynir", "pieces")))

print("+ Rule 3 +")
#   Chaurd == (500 | 1000)
for p in types['pieces']:
    if p != 500 and p != 1000:
        remove_edge("Chaurd", p)
either_or("Chaurd", (500, 1000))

print("+ Rule 4 +")
#   coral reef pieces == Garroda pieces + 250
piece_delta("Garroda", "coral reef", 250)
remove_edge("coral reef", min(neighbors("Garroda", "pieces")))
remove_edge("Garroda", max(neighbors("coral reef", "pieces")))

print("+ Rule 5 +")
remove_edge("city skyline", 1998)
remove_edge("city skyline", 1981)

print("+ Rule 6 +")
#   football pieces == 1981 pieces + 250
piece_delta(1981, "football", 250)

print("+ Rule 7 +")
#   outer space == (Garroda ^ 1250)
#   1992 == (Garroda ^ 1250)
#   Garroda == (outer space ^ 1992)
#   1250 == (outer space ^ 1992)
remove_edge("outer space", 1992)
remove_edge("Garroda", 1250)
either_or("outer space", ("Garroda", 1250))
either_or(1992, ("Garroda", 1250))
either_or("Garroda", ("outer space", 1992))
either_or(1250, ("outer space", 1992))

print("+ Rule 8 +")
#   postage stamp pieces == 1998 pieces + 250
piece_delta(1998, "postage stamp", 250)

print("+ Rule 9 +")
#   Furoth == (1982 ^ 1250)
#   postage stamp == (1982 ^ 1250)
#   1982 == (Furoth ^ postage stamp)
#   1250 == (Furoth ^ postage stamp)
remove_edge("Furoth", "postage stamp")
remove_edge(1982, 1250)
either_or("Furoth", (1982, 1250))
either_or("postage stamp", (1982, 1250))
either_or(1982, ("Furoth", "postage stamp"))
either_or(1250, ("Furoth", "postage stamp"))

print("+ Rule 10 +")
remove_edge("Irycia", 750)

print("+ Rule 11 +")
remove_edge("Rynir", 1999)

print("+ Rule 12 +")
piece_delta("postage stamp", 1999, 1000)

print(f"----- Edges: {G.number_of_edges()}")
print("---- End Rules ----")

# Find any nodes that have a single edge for a specific type
for t, vs in types.items():
    for v in vs:
        for ot in types.keys():
            if ot == t:
                continue
            # print(f"Checking {v} for {ot} edges")
            if has_one_edge(v, ot):
                n1 = v
                n2 = neighbors(v, ot)[0]
                print(f"{n1} has a single '{ot}' edge with {n2}")
                assert n1 in types[t]
                assert n2 in types[ot]
                share_info(n1, n2)


print()
# print(1982, G.adj[1982])
# print(1982, count_edges_per_type(1982))
# print("postage stamp", G.adj["postage stamp"])
# print("postage stamp", count_edges_per_type("postage stamp"))
# print(750, G.adj[750])
# print(750, count_edges_per_type(750))
print(G.number_of_edges())

# nx.draw(G, with_labels=True, font_weight="bold")
# A = nx.nx_agraph.to_agraph(G)
# A.draw(prog="neato")

In [ ]:
print("Rynir", G.adj["Rynir"])
# remove_edge("Rynir", 1500) <-- INVALID - leads to 500 not having a company
remove_edge("Rynir", 1000)

In [ ]:
# remove_edge("outer space", 1250) <-- INVALID - leads to 1250 having no company & no theme
remove_edge("outer space", "Irycia")


In [ ]:
# remove_edge(1250,1992) <-- INVALID - leads to Rynir not having a theme
# remove_edge(1250, "outer space") <-- INVALID - leads to 1250 not having a company or theme

In [ ]:
for item in all_items:
    print(item, count_edges_per_type(item))

In [ ]:
import dash
import dash_cytoscape as cyto
import networkx as nx

# Convert NetworkX graph to Cytoscape JSON format
cytoscape_data = nx.cytoscape_data(G)

# Extract nodes and edges from the Cytoscape data
nodes = [
    {'data': {'id': node['data']['id'], 'label': node['data']['name'], 'color': node['data']['color']}}
    for node in cytoscape_data['elements']['nodes']
]
edges = [
    {'data': {'source': edge['data']['source'], 'target': edge['data']['target'], 
              'id': f"{edge['data']['source']}-{edge['data']['target']}"}}
    for edge in cytoscape_data['elements']['edges']
]

app = dash.Dash(__name__)

default_node_style = {
    'selector': 'node',
    'style': {
        'label': 'data(label)',
        'background-color': '#0074D9',
        'color': '#000000',
        'border-width': '3px',
        'border-color': '#000000',
        "text-wrap": "wrap"
    }
}
default_edge_style = {
    'selector': 'edge',
    'style': {
        'line-color': '#0074D9',
        'width': 2
    }
}

# Create the Dash Cytoscape component
app.layout = dash.html.Div([
    cyto.Cytoscape(
        id='cytoscape-graph',
        elements=nodes + edges,
        layout={'name': 'grid', 'rows': 4, 'cols': 6},
        style={'width': '100%', 'height': '600px'},
        stylesheet = [ default_node_style, default_edge_style,
            {
                'selector': '.highlighted',
                'style': {
                    'background-color': '#FF4136',  # highlighted node color
                    'line-color': '#FF4136',        # highlighted edge color
                    'transition-property': 'background-color, line-color',
                    'transition-duration': '0.5s'
                }
            }
        ],
    ),
    dash.dcc.Store(id='previous-node', data=None)
])

# Callback to handle node clicks
@app.callback(
    dash.Output('cytoscape-graph', 'stylesheet'),
    dash.Input('cytoscape-graph', 'tapNodeData'),
    dash.State('cytoscape-graph', 'stylesheet'),
    prevent_initial_call=True
)
def highlight_node_and_neighbors(node_data, stylesheet):
    if node_data:
        node_id = node_data['id']
        new_styles = [default_node_style, default_edge_style]
        
        # Highlight the selected node and edges
        new_styles.append({
            'selector': f'node[id = "{node_id}"]',
            'style': {
                'background-color': 'red'
            }
        })
        new_styles.append({
            'selector': f'edge[source = "{node_id}"], edge[target = "{node_id}"]',
            'style': {
                'line-color': 'red',
                'width': 3,
            }
        })
        return new_styles
        
    return [default_node_style, default_edge_style]

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
import pprint

pprint.pprint(cytoscape_data)

In [ ]:
for p in pieces:
    print(p, neighbors_by_type(p))
for y in years:
    print(y, neighbors_by_type(y))
for t in themes:
    print(t, neighbors_by_type(t))

In [ ]:
import copy

G_backup = copy.deepcopy(G)